# C7 Style Others

In [3]:
import os
import re
import traceback
import pdfplumber
import PyPDF2
import pandas as pd

# ------------------- Configuration -------------------
BASE_DIRECTORY = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data"
OUTPUT_CSV_PATH_ORIGINAL = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 7/03_raw/ph2_rawdata_cluster7_style_others_originals.csv"
OUTPUT_CSV_PATH_ADDENDUM = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 7/03_raw/ph2_rawdata_cluster7_style_others_addendums.csv"
LOG_FILE_PATH = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 7/03_raw/ph2_scraping_cluster7_style_others_log.txt"
PROJECT_RANGE = range(1007, 1090)   # Original range #(1831, 2193)

# Read the CSV file containing processed projects (with q_id column)
processed_csv_path = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/all_clusters/costs_phase_2_all_clusters_total.csv"  # UPDATE THIS PATH
processed_df = pd.read_csv(processed_csv_path)
# Convert q_id values to numeric then to int for filtering
processed_q_ids = pd.to_numeric(processed_df['q_id'], errors='coerce').dropna().astype(int).unique()
projects_to_process = sorted([q_id for q_id in PROJECT_RANGE if q_id not in processed_q_ids])

# ------------------- Global Tracking Variables -------------------
core_originals = pd.DataFrame()
core_addendums = pd.DataFrame()

scraped_projects = set()
skipped_projects = set()
missing_projects = set()
scraped_pdfs = []
skipped_pdfs = []
addendum_pdfs = []
original_pdfs = []
style_n_pdfs = []  # Not used in this version but kept for consistency

total_pdfs_accessed = 0
total_pdfs_scraped = 0
total_pdfs_skipped = 0

# ------------------- Helper Function for Logging -------------------
def log_msg(msg, log_file):
    """Prints a message to both the log file and console."""
    print(msg, file=log_file)
    print(msg)

# ------------------- Other Helper Functions -------------------
def clean_column_headers(headers):
    """Cleans column headers by normalizing and removing unwanted characters."""
    cleaned_headers = []
    for header in headers:
        if header is None:
            header = ""
        elif isinstance(header, str):
            header = header.lower()
            header = re.sub(r'\s+', ' ', header)
            header = re.sub(r'\(.*?\)', '', header)
            header = re.sub(r'[^a-zA-Z0-9\s]', '', header)
            header = header.strip()
        cleaned_headers.append(header)
    return cleaned_headers

def clean_string_cell(value):
    """Cleans string cells by removing newlines and trimming spaces."""
    if isinstance(value, str):
        return value.replace('\n', ' ').strip()
    elif value is None:
        return ""
    else:
        return str(value).replace('\n', ' ').strip()

def contains_phrase(row, phrase):
    """Checks if any cell in a row contains a specific phrase."""
    regex_pattern = re.sub(r"\s+", r"\\s*", phrase)
    pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
    return row.astype(str).apply(lambda cell: bool(pattern.search(cell))).any()

def extract_specific_phrase(title):
    """
    Extracts a specific phrase from the table title based on predefined keywords.
    """
    phrases = [
        "PTO",
        "Reliability Network Upgrade",
        "Area Delivery Network Upgrade",
        "Local Delivery Network",
        "Other Potential Network Upgrade",
        "Area Delivery Network Upgrades",
        "Conditionally Assigned Network Upgrades",
        "Local Off-Peak Network Upgrade",
        "ADNU",
        "LDNU",
        "RNU"
    ]
    for phrase in phrases:
        if re.search(rf"\b{re.escape(phrase)}\b(?=\d|\W|$)", title, re.IGNORECASE):
            return phrase
    return title  # Fallback if no specific phrase is found

def reorder_columns(df):
    """
    Reorders the columns of the DataFrame based on the specified order.
    """
    desired_order = [
        "q_id",
        "cluster",
        "req_deliverability",
        "latitude",
        "longitude",
        "capacity",
        "point_of_interconnection"
    ]
    existing_desired = [col for col in desired_order if col in df.columns]
    remaining = [col for col in df.columns if col not in existing_desired]
    new_order = existing_desired + remaining
    return df[new_order]

def search_gps_coordinates(text, log_file):
    """Search for GPS coordinates using multiple patterns."""
    gps_coords = re.search(r"gps coordinates:\s*([\d\.\-]+),\s*([\d\.\-]+)", text, re.IGNORECASE)
    if gps_coords:
        log_msg(f"Found GPS coordinates: {gps_coords.groups()}", log_file)
        return gps_coords.groups()
    project_coords = re.search(r"latitude[:\s]*([\d\.\-]+)[^\d]+longitude[:\s]*([\d\.\-]+)", text, re.IGNORECASE)
    if project_coords:
        log_msg(f"Found project coordinates: {project_coords.groups()}", log_file)
        return project_coords.groups()
    gps_coords_directional = re.search(r"gps coordinates:\s*([\d\.\-]+)\s*[nNsS],\s*([\d\.\-]+)\s*[eEwW]", text, re.IGNORECASE)
    if gps_coords_directional:
        lat, lon = gps_coords_directional.groups()
        latitude = lat if "N" in text.upper() else f"-{lat}"
        longitude = lon if "E" in text.upper() else f"-{lon}"
        log_msg(f"Found directional GPS coordinates: {(latitude, longitude)}", log_file)
        return (latitude, longitude)
    log_msg("GPS coordinates not found.", log_file)
    return (None, None)

# ------------------- Appendix PDF Check -------------------
def is_appendix_pdf(pdf_path):
    """Returns True if the first page of the PDF contains 'Appendix A'."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            if not pdf.pages:
                return False
            first_page_text = pdf.pages[0].extract_text() or ""
            return "Appendix A" in first_page_text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return False

# ------------------- Base Data & Table 1 Extraction (Appendix A Only) -------------------

'''
def extract_table1(pdf_path, log_file):
    """
    Extracts the Point of Interconnection from Table 1 in the provided PDF.
    This function is intended to run only on the Appendix A PDF.
    Now it searches for pages containing "Table A.2", "Table B.2" or "Table C.2"
    and within the tables it looks for either "Point of Interconnection" or "POI".
    """
    print(f"\nProcessing {pdf_path} for Table 1 extraction...", file=log_file)
    point_of_interconnection = None
    # Modified to match either "Point of Interconnection" or "POI"
    poi_pattern = re.compile(r"(Point\s+of\s+Interconnection|POI)", re.IGNORECASE)
    table_settings_list = [
        {"horizontal_strategy": "text", "vertical_strategy": "lines", "snap_tolerance": 1},
        {"horizontal_strategy": "lines", "vertical_strategy": "lines", "snap_tolerance": 2}
    ]
    try:
        with pdfplumber.open(pdf_path) as pdf:
            table1_pages = []
            # Modified regex: look for "Table A.2", "Table B.2" or "Table C.2"
            for i, page in enumerate(pdf.pages):
                text = page.extract_text() or ""
                if re.search(r"Table\s*[ABC]\.1\b", text, re.IGNORECASE):
                    table1_pages.append(i)
            if not table1_pages:
                print("No Table 1 found in the PDF.", file=log_file)
                return None
            first_page = table1_pages[0]
            last_page = table1_pages[-1]
            scrape_start = first_page
            scrape_end = last_page + 2
            print(f"Table 1 starts on page {scrape_start + 1} and ends on page {scrape_end + 1}", file=log_file)
            extraction_successful = False
            for page_number in range(scrape_start, min(scrape_end + 1, len(pdf.pages))):
                page = pdf.pages[page_number]
                print(f"\nScraping tables on page {page_number + 1} for Table 1...", file=log_file)
                for attempt, table_settings in enumerate(table_settings_list, start=1):
                    print(f"Attempt {attempt} with settings: {table_settings}", file=log_file)
                    tables = page.find_tables(table_settings=table_settings)
                    print(f"Found {len(tables)} table(s) on page {page_number + 1}", file=log_file)
                    for table_index, table in enumerate(tables, start=1):
                        tab = table.extract()
                        if not tab:
                            print(f"Table {table_index} on page {page_number + 1} is empty.", file=log_file)
                            continue
                        for row_index, row in enumerate(tab, start=1):
                            for cell_index, cell in enumerate(row, start=1):
                                if cell and poi_pattern.search(cell):
                                    poi_col_index = cell_index  # 1-based index
                                    adjacent_col_index = poi_col_index + 1
                                    if adjacent_col_index <= len(row):
                                        poi_value = clean_string_cell(row[adjacent_col_index - 1])
                                        if poi_value:
                                            point_of_interconnection = poi_value
                                            print(f"Found POI: '{point_of_interconnection}' (Page {page_number + 1}, Table {table_index}, Row {row_index})", file=log_file)
                                            extraction_successful = True
                                            break
                                        else:
                                            print(f"POI label found but adjacent value empty (Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                            poi_value_parts = []
                                            current_row_idx = row_index - 1
                                            start_scan = max(0, current_row_idx - 2)
                                            end_scan = min(len(tab), current_row_idx + 3)
                                            for scan_row_index in range(start_scan, end_scan):
                                                if scan_row_index == current_row_idx:
                                                    continue
                                                scan_row = tab[scan_row_index]
                                                if adjacent_col_index - 1 < len(scan_row):
                                                    scan_cell = clean_string_cell(scan_row[adjacent_col_index - 1])
                                                    if scan_cell and not poi_pattern.search(scan_cell):
                                                        poi_value_parts.append(scan_cell)
                                            if poi_value_parts:
                                                point_of_interconnection = " ".join(poi_value_parts)
                                                print(f"Concatenated POI: '{point_of_interconnection}'", file=log_file)
                                                extraction_successful = True
                                                break
                                    else:
                                        print(f"POI label found but no adjacent column (Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                            if extraction_successful:
                                break
                        if extraction_successful:
                            break
                    if extraction_successful:
                        break
                if extraction_successful:
                    break
    except Exception as e:
        print(f"Error processing Table 1 in {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return None
    if not extraction_successful:
        if point_of_interconnection is not None and point_of_interconnection != "":
            print("POI label found but no value.", file=log_file)
            return "Value Missing"
        else:
            print("POI not found in Table 1.", file=log_file)
            return None
    return point_of_interconnection
'''

import re
import traceback
import pdfplumber

def extract_table1(pdf_path, log_file):
    """
    Extracts the Point of Interconnection from Table A.1/B.1/C.1 in the provided PDF.
    Prefers a row labeled exactly "POI" if present; otherwise behaves exactly as your old code.
    Returns the string in `point_of_interconnection`, or None/“Value Missing” as before.
    """
    print(f"\nProcessing {pdf_path} for Table 1 extraction...", file=log_file)
    point_of_interconnection = None

    poi_pattern       = re.compile(r"(Point\s+of\s+Interconnection|POI)", re.IGNORECASE)
    exact_poi_pattern = re.compile(r"^\s*POI\s*$", re.IGNORECASE)

    table_settings_list = [
        {"horizontal_strategy": "text",  "vertical_strategy": "lines", "snap_tolerance": 1},
        {"horizontal_strategy": "lines", "vertical_strategy": "lines", "snap_tolerance": 2},
    ]

    candidates = []        # label, value, page#, table#, row#
    extraction_successful = False

    try:
        with pdfplumber.open(pdf_path) as pdf:
            # find Table A.1/B.1/C.1 pages
            table1_pages = []
            for i, page in enumerate(pdf.pages):
                text = page.extract_text() or ""
                if re.search(r"Table\s*[ABC]\.1\b", text, re.IGNORECASE):
                    table1_pages.append(i)

            if not table1_pages:
                print("No Table A.1/B.1/C.1 found in the PDF.", file=log_file)
                return None

            start = table1_pages[0]
            end   = min(table1_pages[-1] + 2, len(pdf.pages) - 1)
            print(f"Table 1 pages: {start+1}–{end+1}", file=log_file)

            # scan pages
            for page_number in range(start, end + 1):
                page = pdf.pages[page_number]
                print(f"\nScanning page {page_number+1}…", file=log_file)

                for attempt, settings in enumerate(table_settings_list, start=1):
                    print(f"  Attempt {attempt} with {settings}", file=log_file)
                    tables = page.find_tables(table_settings=settings)
                    print(f"    Found {len(tables)} table(s)", file=log_file)

                    for table_index, table in enumerate(tables, start=1):
                        tab = table.extract()
                        if not tab:
                            print(f"    Table {table_index} empty—skipping.", file=log_file)
                            continue

                        for row_index, row in enumerate(tab, start=1):
                            for cell_index, cell in enumerate(row, start=1):
                                if cell and poi_pattern.search(cell):
                                    label = cell.strip()
                                    # adjacent cell
                                    poi_value = None
                                    if cell_index < len(row):
                                        poi_value = clean_string_cell(row[cell_index]).strip() or None

                                    # concatenation fallback
                                    if poi_value is None:
                                        parts = []
                                        cur = row_index - 1
                                        lo = max(0, cur - 2)
                                        hi = min(len(tab), cur + 3)
                                        for r2 in range(lo, hi):
                                            if r2 == cur: continue
                                            row2 = tab[r2]
                                            if cell_index < len(row2):
                                                txt = clean_string_cell(row2[cell_index]).strip()
                                                if txt and not poi_pattern.search(txt):
                                                    parts.append(txt)
                                        if parts:
                                            poi_value = " ".join(parts)

                                    candidates.append((label, poi_value, page_number+1, table_index, row_index))
                                    if poi_value:
                                        extraction_successful = True
                            # end cell loop
                        # end row loop
                    # end table loop
                # end settings loop
            # end page loop

    except Exception as e:
        print(f"Error processing Table 1 in {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return None

    # no candidates
    if not candidates:
        print("POI not found in Table 1.", file=log_file)
        return None

    # 1) exact “POI” label
    for label, val, pg, tb, rw in candidates:
        if exact_poi_pattern.match(label):
            point_of_interconnection = val
            print(f"Exact‐match POI → '{val}' (Page {pg}, Table {tb}, Row {rw})", file=log_file)
            return point_of_interconnection

    # 2) first non‐null as before
    for label, val, pg, tb, rw in candidates:
        if val:
            point_of_interconnection = val
            print(f"Found POI: '{val}' (from label '{label}')", file=log_file)
            return point_of_interconnection

    # 3) label found but empty value
    print("POI label found but no value.", file=log_file)
    return "Value Missing"


def extract_base_data(pdf_path, project_id, log_file):
    """
    Extracts base data from the Appendix A PDF.
    (This function is meant to run only on a PDF verified as an Appendix A PDF.)
    """
    if not is_appendix_pdf(pdf_path):
        log_msg(f"Skipping base extraction because {pdf_path} is not an Appendix A PDF.", log_file)
        return pd.DataFrame()
    log_msg("Extracting base data from Appendix A PDF...", log_file)
    try:
        with open(pdf_path, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text
        text = clean_string_cell(text)
        #queue_id = re.search(r"q[\s_-]*(\d+)", text, re.IGNORECASE)
        #queue_id = queue_id.group(1) if queue_id else str(project_id)
        queue_id = str(project_id)
        log_msg(f"Extracted Queue ID: {queue_id}", log_file)
        #clusters = re.findall(r"queue[\s_-]*cluster[\s_-]*(\d+)", text, re.IGNORECASE)
        #if '7' in clusters:
        #    cluster_number = '7'
        #elif clusters:
        #    cluster_number = max(clusters, key=lambda x: int(x))
        #else:
        #   cluster_number = '7'
        clusters = re.findall(r"queue[\s_-]*cluster[\s_-]*(\d+)", text, re.IGNORECASE)
        cluster_number = '7'    
        log_msg(f"Extracted Cluster Number: {cluster_number}", log_file)
        deliverability_status = re.search(r"(\w+)\s*capacity deliverability status", text, re.IGNORECASE)
        deliverability_status = deliverability_status.group(1) if deliverability_status else None
        log_msg(f"Extracted Deliverability Status: {deliverability_status}", log_file)
        capacity = re.search(r"total rated output of (\d+)\s*mw", text, re.IGNORECASE)
        if capacity:
            capacity = int(capacity.group(1))
        else:
            capacity2 = re.search(r"(\d+)\s*mw", text)
            capacity = int(capacity2.group(1)) if capacity2 else None
        log_msg(f"Extracted Capacity: {capacity}", log_file)
        poi_value = extract_table1(pdf_path, log_file)
        latitude, longitude = search_gps_coordinates(text, log_file)
        base_data = {
            "q_id": [queue_id],
            "cluster": [cluster_number],
            "req_deliverability": [deliverability_status],
            "latitude": [latitude],
            "longitude": [longitude],
            "capacity": [capacity],
            "point_of_interconnection": [poi_value]
        }
        log_msg("Base data extracted:", log_file)
        log_msg(str(base_data), log_file)
        return pd.DataFrame(base_data)
    except Exception as e:
        log_msg(f"Error extracting base data from {pdf_path}: {e}", log_file)
        print(traceback.format_exc(), file=log_file)
        return pd.DataFrame()

# ------------------- Attachment 2 Processing & Merging -------------------
def is_attachment2_pdf(pdf_path):
    """
    Returns True if the first page of the PDF contains something like
    "Attachment 2", "Attachment #2", "Attachement 2", or "Attachment2".
    """
    # regex explanation:
    #   \b            start on a word boundary
    #   Attache?ment  matches "Attachment" or the common typo "Attachement"
    #   \s*           allow any amount of whitespace (including none)
    #   #?            optional '#'
    #   \s*           optional whitespace again
    #   2\b           the digit 2, ending on a word boundary
    pattern = re.compile(r"\bAttache?ment\s*#?\s*2\b", flags=re.IGNORECASE)

    try:
        with pdfplumber.open(pdf_path) as pdf:
            if not pdf.pages:
                return False
            text = pdf.pages[0].extract_text() or ""
            return bool(pattern.search(text))
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return False
    
def is_addendum_pdf(pdf_path):
    """
    Returns True if the first page of the PDF contains "Addendum" or "Revision".
    """
    try:
        with pdfplumber.open(pdf_path) as pdf:
            if not pdf.pages:
                return False
            first_page_text = pdf.pages[0].extract_text() or ""
            return ("Addendum" in first_page_text) or ("Revision" in first_page_text)
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return False

def make_unique_headers(headers):
    """
    Appends a suffix to duplicate headers to make them unique.
    """
    seen = {}
    unique_headers = []
    for header in headers:
        if header in seen:
            seen[header] += 1
            unique_headers.append(f"{header}_{seen[header]}")
        else:
            seen[header] = 1
            unique_headers.append(header)
    return unique_headers



'''

def extract_first_table(pdf_path, log_file):
    """
    Iterates over all pages and returns the first non-empty table found as a DataFrame,
    treating the first row as the column headers and ensuring unique header names.
    """
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                tables = page.extract_tables()
                for table in tables:
                    if table and any(any(cell is not None and cell.strip() for cell in row) for row in table):
                        df = pd.DataFrame(table)
                        if not df.empty:
                            # Use the first row as headers
                            headers = df.iloc[0].tolist()
                            headers = make_unique_headers(headers)
                            df.columns = headers
                            df = df[1:].reset_index(drop=True)
                            # Remove any duplicate column names (safeguard)
                            df = df.loc[:, ~df.columns.duplicated()]
                            return df
    except Exception as e:
        log_msg(f"Error extracting table from {pdf_path}: {e}",log_file)
    return pd.DataFrame()

'''

def extract_first_table(pdf_path, log_file):
    """
    Extracts the *first* non-empty table from the entire PDF (just like before).
    Then, on the *same page* where that first table was found, searches for another
    table that contains the phrase "Other Potential" in any cell. If found, merges
    that second table with the first. Returns the resulting DataFrame.

    NOTE:
      - Currently, this merges the two tables *row-wise* (one below the other).
      - If you prefer side-by-side merging (assuming same number of rows), replace
        the pd.concat(..., axis=0) with pd.concat(..., axis=1).
      - If you want to handle more complex scenarios (e.g., multiple "Other Potential"
        tables or table on a subsequent page), you will need to expand the logic.
    """
    import pdfplumber
    import pandas as pd
    
    first_table_df = pd.DataFrame()
    found_page_index = None

    try:
        # 1) Find the *first* non-empty table in the PDF
        with pdfplumber.open(pdf_path) as pdf:
            for page_index, page in enumerate(pdf.pages):
                tables = page.extract_tables()
                for table in tables:
                    if table and any(any(cell and cell.strip() for cell in row) for row in table):
                        # Convert list-of-lists to a DataFrame
                        df = pd.DataFrame(table)
                        if not df.empty:
                            # Use the first row as headers

                            headers = df.iloc[0].tolist()
                            headers = make_unique_headers(headers)
                            df.columns = headers
                            df = df[1:].reset_index(drop=True)
                            # Remove any duplicate column names (safeguard)
                            df = df.loc[:, ~df.columns.duplicated()]
                             
                             
                            
                            first_table_df = df
                            found_page_index = page_index
                            break  # Stop looking at more tables on this page
                if found_page_index is not None:
                    # We found our first table; stop searching further pages
                    break

            # If no table was found at all, just return empty DataFrame
            if first_table_df.empty:
                return first_table_df

            # 2) On that *same page*, look for a table containing "Other Potential"
            other_table_df = pd.DataFrame()
            if found_page_index is not None:
                page = pdf.pages[found_page_index]
                tables = page.extract_tables()
                
                for table in tables:
                    if table and any(any(cell and cell.strip() for cell in row) for row in table):
                        # Check if "Other Potential" appears in any cell
                        found_phrase = False
                        for row in table:
                            for cell in row:
                                if cell and "other potential" in cell.lower():
                                    found_phrase = True
                                    break
                            if found_phrase:
                                break

                        # If found, convert to DataFrame
                        if found_phrase:
                            df = pd.DataFrame(table)
                            if not df.empty:
                               
                                headers = df.iloc[0].tolist()
                                headers = make_unique_headers(headers)
                                df.columns = headers
                                df = df[1:].reset_index(drop=True)
                                # Remove any duplicate column names (safeguard)
                                df = df.loc[:, ~df.columns.duplicated()]
                                other_table_df = df
                                break  # Stop at the first "Other Potential" table

            # 3) Merge the two tables if the second one was found
            if other_table_df.empty:
                # No "Other Potential" table found; return only the first table
                return first_table_df
            else:
                # Example: Merge *row-wise* (append below). 
                # If you prefer side-by-side (and row counts match), use axis=1.
                # ---- Merge duplicate columns before final concatenation ----
                common_cols = set(first_table_df.columns).intersection(other_table_df.columns)
                for col in common_cols:
                    # Combine (fill missing in first_table_df with data from other_table_df)
                    first_table_df[col] = first_table_df[col].combine_first(other_table_df[col])
                    # Remove the duplicate column from the second table
                    other_table_df.drop(columns=[col], inplace=True)

                merged_df = pd.concat([first_table_df, other_table_df], ignore_index=True)
                return merged_df

    except Exception as e:
        print(f"Error extracting table from {pdf_path}: {e}", file=log_file)
        return pd.DataFrame()



'''
def extract_first_table(pdf_path,log_file):
    """
    Iterates over all pages and returns the first non-empty table found as a DataFrame.
    It first extracts the table normally (using the first row as header).
    Then it scans the raw table rows for a row that, after replacing newline characters with a space,
    has a contiguous block of cells (starting at index 1) exactly equal to:
        ["Costs per Category w/o ITCC (A)",
         "One Time Costs (B)",
         "Total Costs w/o ITCC (C=A+B)",
         "Total Escalated Costs w/o ITCC",
         "Estimated Time for Licensing, Permitting, & Construction (Months)",
         "Maximum Escalation Duration (Months)"]
    If found, it discards all rows above that row, uses that row as the header (taking columns 1 to 6),
    and returns the resulting DataFrame. Otherwise, it returns the table as originally extracted.
    """
    import pdfplumber, pandas as pd

    # Define the expected header row exactly as you want it.
    expected_header = [
        "Costs per Category w/o ITCC (A)",
        "One Time Costs (B)",
        "Total Costs w/o ITCC (C=A+B)",
        "Total Escalated Costs w/o ITCC",
        "Estimated Time for Licensing, Permitting, & Construction (Months)",
        "Maximum Escalation Duration (Months)"
    ]
    
    # First, extract the raw table from the PDF.
    try:
        with pdfplumber.open(pdf_path) as pdf:
            raw_table = None
            for page in pdf.pages:
                tables = page.extract_tables()
                for table in tables:
                    if table and any(any(cell is not None and cell.strip() for cell in row) for row in table):
                        raw_table = table
                        break
                if raw_table is not None:
                    break
    except Exception as e:
        print(f"Error extracting table from {pdf_path}: {e}")
        return pd.DataFrame()
    
    if raw_table is None:
        return pd.DataFrame()
    
    # Build a DataFrame from the raw table (without assigning header)
    df_raw = pd.DataFrame(raw_table)
    if df_raw.empty or len(df_raw) < 2:
        return pd.DataFrame()
    
    # Also build a fallback DataFrame using the first row as header
    df_fallback = df_raw.copy()
    fallback_header = df_fallback.iloc[0].tolist()
    df_fallback.columns = fallback_header
    df_fallback = df_fallback.iloc[1:].reset_index(drop=True)
    
    # Now, scan each row of the raw table to see if it contains the expected header.
    # We replace newline characters with a space for the comparison.
    special_idx = None
    for i in range(len(df_raw)):
        row_vals = df_raw.iloc[i].tolist()
        cleaned = [cell.replace("\n", " ") if cell is not None else "" for cell in row_vals]
        # Check if the row has at least 1 + len(expected_header) cells
        if len(cleaned) >= 1 + len(expected_header):
            # Compare cells 1 to 1+len(expected_header)
            if cleaned[1:1+len(expected_header)] == expected_header:
                special_idx = i
                break

    if special_idx is not None:
        # Found the special header row.
        # Remove all rows above that row.
        df_new = df_raw.iloc[special_idx:].reset_index(drop=True)
        # Use columns 1 to 1+len(expected_header) from that row as header.
        new_header = df_new.iloc[0].tolist()[1:1+len(expected_header)]
        log_msg(f"Special header found at row {special_idx}. New header: {new_header}", log_file)
        # Now, select only those columns.
        df_new = df_new.iloc[:, 1:1+len(expected_header)]
        # Drop the header row from data.
        df_new = df_new.iloc[1:].reset_index(drop=True)
        df_new.columns = new_header
        return df_new
    else:
        log_msg("No special header row found. Using fallback header.", log_file)
        return df_fallback
'''


def update_base_data(existing_df, new_df):
    """
    For each column in existing_df (assumed to be a single-row DataFrame),
    if the value is missing (empty string, "None", or NA), update it with the corresponding
    value from new_df (if provided and not missing).
    Returns the updated DataFrame.
    """
    for col in existing_df.columns:
        existing_val = existing_df.at[0, col]
        # Use pd.isna() to check for NA values.
        if pd.isna(existing_val) or existing_val == "" or existing_val == "None":
            if col in new_df.columns:
                new_val = new_df.at[0, col]
                if not (pd.isna(new_val) or new_val == "" or new_val == "None"):
                    existing_df.at[0, col] = new_val
    return existing_df



def process_attachment2_for_project(project_id, log_file):
    """
    For the given project:
      1. Identify all original Appendix A PDFs (non‑revision) in the project's "03_phase_2_study" folder.
      2. If none are found, build a minimal base_data_df with only q_id (others None).
      3. Otherwise extract base data from the first Appendix A and fill missing from the rest.
      4. Process addendum Appendix A similarly.
      5. Scrape Attachment 2 PDFs and merge with whichever base_data_df you have.
    """
    global total_pdfs_accessed, total_pdfs_scraped, total_pdfs_skipped
    project_folder = os.path.join(BASE_DIRECTORY, str(project_id), "03_phase_2_study")
    if not os.path.exists(project_folder):
        log_msg(f"Project folder not found: {project_folder}", log_file)
        return pd.DataFrame(), pd.DataFrame()

    # 1) find Appendix A PDFs
    original_appendix_pdfs = []
    addendum_appendix_pdf = None
    for f in os.listdir(project_folder):
        if not f.lower().endswith(".pdf"):
            continue
        pdf_path = os.path.join(project_folder, f)
        if is_appendix_pdf(pdf_path):
            if is_addendum_pdf(pdf_path) and addendum_appendix_pdf is None:
                addendum_appendix_pdf = f
            elif not is_addendum_pdf(pdf_path):
                original_appendix_pdfs.append(f)

    # 2) find Attachment 2 PDFs
    attachment2_pdfs = [
        f for f in os.listdir(project_folder)
        if f.lower().endswith(".pdf") and is_attachment2_pdf(os.path.join(project_folder, f))
    ]


    # 1.2) if neither Appendix A nor Attachment 2 → skip entirely
    if not original_appendix_pdfs and not attachment2_pdfs:
        log_msg(f"No Appendix A and no Attachment 2 for project {project_id}. Skipping.", log_file)
        skipped_projects.add(project_id)
        return pd.DataFrame(), pd.DataFrame()      


     

    # 2) build base_data_df
    if not original_appendix_pdfs:
        if addendum_appendix_pdf:
            # ── new: use addendum as the base
            log_msg(f"No original Appendix A PDF found for project {project_id}. Using addendum for base data.", log_file)
            path = os.path.join(project_folder, addendum_appendix_pdf)
            base_data_df = extract_base_data(path, project_id, log_file)
            if base_data_df.empty:
                log_msg(f"--> Addendum scrape yielded nothing; falling back to minimal base.", log_file)
                base_data_df = pd.DataFrame({
                    "q_id":                   [str(project_id)],
                    "cluster":                [None],
                    "req_deliverability":     [None],
                    "latitude":               [None],
                    "longitude":              [None],
                    "capacity":               [None],
                    "point_of_interconnection":[None]
                })
    else:
        # extract from first Appendix A
        base_pdf = original_appendix_pdfs[0]
        path = os.path.join(project_folder, base_pdf)
        log_msg(f"Scraped base data from original Appendix A PDF: {path}", log_file)
        base_data_df = extract_base_data(path, project_id, log_file)

        # fill any missing fields from other originals
        missing = [c for c in base_data_df.columns
                   if pd.isna(base_data_df.at[0,c]) or base_data_df.at[0,c] in ["", "None"]]
        if missing:
            log_msg(f"Missing base columns {missing}—attempting updates", log_file)
            for other in original_appendix_pdfs[1:]:
                new_df = extract_base_data(os.path.join(project_folder, other), project_id, log_file)
                base_data_df = update_base_data(base_data_df, new_df)
                missing = [c for c in base_data_df.columns
                           if pd.isna(base_data_df.at[0,c]) or base_data_df.at[0,c] in ["", "None"]]
                if not missing:
                    log_msg("Successfully filled all missing base data.", log_file)
                    break
            if missing:
                log_msg(f"Still missing after updates: {missing}", log_file)

    # 3) prepare addendum base_data_df
    if addendum_appendix_pdf and original_appendix_pdfs:
        add_path = os.path.join(project_folder, addendum_appendix_pdf)
        log_msg(f"Scraped base data from addendum Appendix A PDF: {add_path}", log_file)
        addendum_base_data_df = extract_base_data(add_path, project_id, log_file)
        if addendum_base_data_df.empty:
            addendum_base_data_df = base_data_df.copy()
    else:
        addendum_base_data_df = base_data_df.copy()

    # 4) now scrape Attachment 2 tables
    attachment_data_list = []
    attachment_addendum_list = []
    for f in os.listdir(project_folder):
        if not f.lower().endswith(".pdf"):
            continue
        pdf_path = os.path.join(project_folder, f)
        log_msg(f"Accessing PDF: {pdf_path}", log_file)
        total_pdfs_accessed += 1

        if is_attachment2_pdf(pdf_path):
            log_msg(f"Scraping Attachment 2 PDF: {pdf_path}", log_file)
            table_df = extract_first_table(pdf_path, log_file)
            if table_df.empty:
                log_msg(f"--> No table found in {f}, skipping.", log_file)
                skipped_pdfs.append(f)
                total_pdfs_skipped += 1
                continue

            # avoid collisions
            for col in set(base_data_df.columns) & set(table_df.columns):
                table_df.rename(columns={col: f"{col}_table"}, inplace=True)

            # choose base vs addendum
            if is_addendum_pdf(pdf_path):
                base_df = addendum_base_data_df
                addendum_pdfs.append(f)
            else:
                base_df = base_data_df
                original_pdfs.append(f)

            # merge side‑by‑side
            repeated = pd.concat([base_df] * len(table_df), ignore_index=True)
            merged = pd.concat([repeated, table_df.reset_index(drop=True)], axis=1)

            if is_addendum_pdf(pdf_path):
                attachment_addendum_list.append(merged)
            else:
                attachment_data_list.append(merged)

            scraped_pdfs.append(f)
            total_pdfs_scraped += 1
        else:
            log_msg(f"--> Not an Attachment 2 PDF: skipping {f}", log_file)
            skipped_pdfs.append(f)
            total_pdfs_skipped += 1

    # 5) record project status
    if not attachment_data_list and not attachment_addendum_list:
        skipped_projects.add(project_id)
    else:
        scraped_projects.add(project_id)

    


    project_attachment_df = pd.concat(attachment_data_list, ignore_index=True) if attachment_data_list else pd.DataFrame()
    project_attachment_addendum_df = pd.concat(attachment_addendum_list, ignore_index=True) if attachment_addendum_list else pd.DataFrame()
    return project_attachment_df, project_attachment_addendum_df


# ------------------- CSV Saving & Summary Functions -------------------
def save_to_csv(df, output_csv_path, data_type):
    """Cleans the DataFrame and saves it to a CSV file."""
    if df.empty:
        print(f"No data to save for {data_type}.")
        return
    df = df.applymap(clean_string_cell)
    df = df[~df.apply(lambda row: contains_phrase(row, "Type of Upgrade"), axis=1)]
    df = reorder_columns(df)
    print(f"\nColumns reordered for {data_type} as per specification.")
    if 'q_id' in df.columns:
        df['q_id'] = pd.to_numeric(df['q_id'], errors='coerce')
    try:
        df.to_csv(output_csv_path, index=False)
        print(f"\nData successfully saved to {output_csv_path}")
    except Exception as e:
        print(f"Error saving {data_type} data to CSV: {e}")
        print(traceback.format_exc())

# ------------------- Main Processing Function -------------------
def process_pdfs_in_folder():
    """
    Processes all projects in ascending order (filtered via projects_to_process).
    For each project, it:
      - Checks for the project folder.
      - Processes Attachment 2 PDFs by merging base data (from Appendix A PDFs) with
        scraped table data (only the first nonempty table).
      - Aggregates results across projects.
    After processing, it saves the combined results to CSV files and prints a summary.
    """
    global core_originals, core_addendums, total_pdfs_accessed, total_pdfs_scraped, total_pdfs_skipped
    all_attachment_data = []
    all_attachment_addendum_data = []

    os.makedirs(os.path.dirname(LOG_FILE_PATH), exist_ok=True)
    with open(LOG_FILE_PATH, 'w') as log_file:
        log_msg(f"Projects to process: {projects_to_process}", log_file)
        for project_id in projects_to_process:
            project_folder = os.path.join(BASE_DIRECTORY, str(project_id))
            if not os.path.exists(project_folder):
                missing_projects.add(project_id)
                log_msg(f"Project folder not found: {project_folder}", log_file)
                continue
            log_msg(f"\n--- Processing project {project_id} ---", log_file)
            proj_attach_df, proj_attach_add_df = process_attachment2_for_project(project_id, log_file)
            if not proj_attach_df.empty:
                all_attachment_data.append(proj_attach_df)
            if not proj_attach_add_df.empty:
                all_attachment_addendum_data.append(proj_attach_add_df)
        
        if all_attachment_data:
            core_originals = pd.concat(all_attachment_data, ignore_index=True)
            save_to_csv(core_originals, OUTPUT_CSV_PATH_ORIGINAL, "originals")
        else:
            log_msg("\nNo Attachment 2 data processed for regular PDFs.", log_file)
        
        if all_attachment_addendum_data:
            core_addendums = pd.concat(all_attachment_addendum_data, ignore_index=True)
            save_to_csv(core_addendums, OUTPUT_CSV_PATH_ADDENDUM, "addendums")
        else:
            log_msg("\nNo Attachment 2 data processed for addendum PDFs.", log_file)

        total_projects_processed = len(scraped_projects) + len(skipped_projects)
        log_msg("\n=== Scraping Summary ===", log_file)
        log_msg(f"Total Projects Processed: {total_projects_processed}", log_file)
        log_msg(f"Total Projects Scraped: {len(scraped_projects)}", log_file)
        log_msg(f"Total Projects Skipped: {len(skipped_projects)}", log_file)
        log_msg(f"Total Projects Missing: {len(missing_projects)}", log_file)
        log_msg(f"Total PDFs Accessed: {total_pdfs_accessed}", log_file)
        log_msg(f"Total PDFs Scraped: {total_pdfs_scraped}", log_file)
        log_msg(f"Total PDFs Skipped: {total_pdfs_skipped}", log_file)
        log_msg("\nList of Scraped Projects: " + str(sorted(scraped_projects)), log_file)
        log_msg("\nList of Skipped Projects: " + str(sorted(skipped_projects)), log_file)
        log_msg("\nList of Missing Projects: " + str(sorted(missing_projects)), log_file)
        log_msg("\nList of Scraped PDFs: " + str(scraped_pdfs), log_file)
        log_msg("\nList of Skipped PDFs: " + str(skipped_pdfs), log_file)
        log_msg("\nList of Addendum PDFs: " + str(addendum_pdfs), log_file)
        log_msg("\nList of Original PDFs: " + str(original_pdfs), log_file)
        log_msg("\nList of Style N PDFs (Skipped due to 'Network Upgrade Type'): " + str(style_n_pdfs), log_file)
        log_msg("\nTotal Number of Style N PDFs: " + str(len(style_n_pdfs)), log_file)
        log_msg("\nNumber of Original PDFs Scraped: " + str(len([pdf for pdf in scraped_pdfs if pdf in original_pdfs])), log_file)
        log_msg("Number of Addendum PDFs Scraped: " + str(len([pdf for pdf in scraped_pdfs if pdf in addendum_pdfs])), log_file)

# ------------------- Main -------------------
def main():
    process_pdfs_in_folder()

if __name__ == "__main__":
    main()


<>:151: SyntaxWarning: invalid escape sequence '\s'
<>:151: SyntaxWarning: invalid escape sequence '\s'
/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_67294/354511407.py:151: SyntaxWarning: invalid escape sequence '\s'
  '''


Projects to process: [1007, 1008, 1009, 1012, 1013, 1015, 1017, 1018, 1019, 1020, 1022, 1023, 1024, 1025, 1026, 1034, 1035, 1037, 1039, 1041, 1042, 1043, 1044, 1046, 1049, 1051, 1052, 1054, 1055, 1056, 1057, 1059, 1060, 1061, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089]

--- Processing project 1007 ---
No Appendix A and no Attachment 2 for project 1007. Skipping.
Project folder not found: /Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1008
Project folder not found: /Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1009
Project folder not found: /Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1012

--- Processing project 1013 ---
No Appendix A and no Attachment 2 for project 1013. Skipping.

--- Processing project 1015 ---
No Appendix A and no Attachment 2 for project 1015. Skipping.
Project folder not fo

/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_67294/354511407.py:896: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(clean_string_cell)


# Itemized and Addendums

# Originals

# Column names

In [4]:
import pandas as pd
import re
import unicodedata

# Load the CSV file
df = pd.read_csv("/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 7/03_raw/ph2_rawdata_cluster7_style_others_originals.csv", dtype={'estimated_time_to_construct': str})

df['q_id'] = df['q_id'].astype('Int64')
df['cluster'] = df['cluster'].astype('Int64')


def clean_column_headers(headers):
    """Cleans column headers by normalizing and removing unwanted characters."""
    cleaned_headers = []  # Initialize an empty list to hold the cleaned header names.
    for header in headers:  # Iterate over each header in the input.
        if header is None:
            header = ""  # If the header is None, set it to an empty string.
        elif isinstance(header, str):  # Otherwise, if the header is a string:
            #header = header.lower()  # Convert the header to lowercase.
            header = re.sub(r'\s+', ' ', header)  # Replace one or more whitespace characters with a single space.
            #header = re.sub(r'\(.*?\)', '', header)  # Remove any text within parentheses (non-greedy).
            header = re.sub(r'[^a-zA-Z0-9\s()/+=_]', '', header)  # Remove any character that is not a letter, number, or whitespace.
            header = header.strip()  # Remove any leading or trailing whitespace.
        cleaned_headers.append(header)  # Append the cleaned header to the list.
    return cleaned_headers  # Return the list of cleaned headers.



df.columns = clean_column_headers(df.columns)

def convert_to_snake_case(column_name):
    column_name = column_name.strip().lower()
    column_name = re.sub(r'[\s\-]+', '_', column_name)
    column_name = re.sub(r'[^\w]', '', column_name)
    return column_name

def clean_string_cell(value):
    if isinstance(value, str):
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
        value = value.replace('\n', ' ').strip()
    return value

df = df.map(clean_string_cell)
df.columns = [convert_to_snake_case(col) for col in df.columns]
print("After cleaning:", df.columns.tolist())

After cleaning: ['q_id', 'cluster', 'req_deliverability', 'latitude', 'longitude', 'capacity', 'point_of_interconnection', 'project_q1064tot728afs', 'unnamed_8', 'none_2', 'none_3', 'none_4', 'none_5', 'none_6', 'none_7', 'element', 'interconnection_facilities_costs_x_1000_constant_dollar_2015', 'reliability_network_upgrades_costs_x_1000_constant_dollar_2015', 'delivery_network_upgrades_costs_x_1000_constant_dollar_2015', 'distribution_upgrades_costs_x_1000_constant_dollar_2015', 'total_estimated_costs_x_1000_constant_dollar_2015', 'total_estimated_costs_x_1000_escalated_constant_dollars_od_year', 'estimated_time_to_construct_months_note_345_9_10', 'od_dollar_escalation_duration_months_note_345_9_10', 'project_q1065tot729afs', 'project_q1066tot730afs', 'project_q1069tot714', 'project_q1070tot708', 'project_q1071tot721', 'project_q1074tot727', 'project_q1076tot726']


# Itemized

In [9]:
import pandas as pd
import re
import unicodedata
import numpy as np
# Load the CSV file
df = pd.read_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 7/03_raw/ph2_rawdata_cluster7_style_others_originals.csv', dtype={'estimated_time_to_construct': str})

#df['q_id'] = df['q_id'].astype('Int64')
df['cluster'] = df['cluster'].astype('Int64')




def clean_column_headers(headers):
    """Cleans column headers by normalizing and removing unwanted characters."""
    cleaned_headers = []  # Initialize an empty list to hold the cleaned header names.
    for header in headers:  # Iterate over each header in the input.
        if header is None:
            header = ""  # If the header is None, set it to an empty string.
        elif isinstance(header, str):  # Otherwise, if the header is a string:
            #header = header.lower()  # Convert the header to lowercase.
            header = re.sub(r'\s+', ' ', header)  # Replace one or more whitespace characters with a single space.
            #header = re.sub(r'\(.*?\)', '', header)  # Remove any text within parentheses (non-greedy).
            header = re.sub(r'[^a-zA-Z0-9\s()/+=_]', '', header)  # Remove any character that is not a letter, number, or whitespace.
            header = header.strip()  # Remove any leading or trailing whitespace.
        cleaned_headers.append(header)  # Append the cleaned header to the list.
    return cleaned_headers  # Return the list of cleaned headers.

df.columns = clean_column_headers(df.columns)

 



#STEP 2: NAMING CONVENTION
def convert_to_snake_case(column_name):
    column_name = column_name.strip().lower()
    column_name = re.sub(r'[\s\-]+', '_', column_name)
    column_name = re.sub(r'[^\w]', '', column_name)
    return column_name

def clean_string_cell(value):
    if isinstance(value, str):
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
        value = value.replace('\n', ' ').strip()
    return value

df = df.map(clean_string_cell)
df.columns = [convert_to_snake_case(col) for col in df.columns]




def move_dollar_values(df, source_column, target_column):
    """
    For each row in the DataFrame, if the value in `source_column` starts with a '$',
    move that value to `target_column` and clear the value in the source column.

    Parameters:
      df (pd.DataFrame): The input DataFrame.
      source_column (str): The column to check for values starting with '$'.
      target_column (str): The column to move the values into.

    Returns:
      pd.DataFrame: The modified DataFrame.
    """
    # Ensure target_column exists; if not, create it filled with empty strings.
    if target_column not in df.columns:
        df[target_column] = ""
    
    # Create a boolean mask for rows where the source column starts with '$'
    mask = df[source_column].astype(str).str.startswith('$', na=False)
    
    # Move the values: assign the source values to the target column where the mask is True.
    df.loc[mask, target_column] = df.loc[mask, source_column]
    
    # Clear the source column values for those rows (set to empty string)
    df.loc[mask, source_column] = ""
    
    return df

# Move values from 'unnamed_8' to a new column 'moved_value'
#df = move_dollar_values(df, 'none_5', 'total_estimated_costs_x_1000_escalated_constant_dollars_od_year')


#df = move_dollar_values(df, 'none_3','total_estimated_costs_x_1000_constant_dollar_2020')

def remove_dollar_values_and_fill_nan(df, column):
    """
    For each row in the DataFrame, if the value in the specified column starts with '$',
    set that value to NaN. Also, replace any empty strings in that column with NaN.

    Parameters:
      df (pd.DataFrame): The input DataFrame.
      column (str): The column to check and clean.

    Returns:
      pd.DataFrame: The modified DataFrame.
    """
    # Ensure the column is treated as string
    df[column] = df[column].astype(str)
    
    # Set values starting with '$' to NaN
    mask = df[column].str.startswith('$', na=False)
    df.loc[mask, column] = np.nan
    
    # Replace any remaining empty strings with NaN
    df[column] = df[column].replace("", np.nan)
    
    return df

df = remove_dollar_values_and_fill_nan(df, 'unnamed_8')


df = df[~df.apply(lambda row: any(str(cell).startswith("Costs per Category w/o ITCC (A)") for cell in row), axis=1)]
 


def merge_columns(df):

    merge_columns_dict = {

 

        "one_time_costs_b": [
            "none_2",
        ],

 
    
        "type_of_upgrade": [
            "cost_ca_tegory",
            "cost_category",
            "cost_category_notes_1a_to_1f",
           
            'other_potential_cost', 
            'other_potential_network_cost',
            'other_potential_network_upgrades_note_1112',
          'other_potential_network_upgrades', 
          'project_q1064tot728afs',
          'project_q1065tot729afs', 'project_q1066tot730afs', 'project_q1069tot714', 'project_q1070tot708', 'project_q1071tot721', 'project_q1074tot727', 'project_q1076tot726', 'element', 
          'other_potential_network_upgrades_note_1112', 'project_q1219tot752',
          
          
           
           
           
           
            
        ],
        "escalated_cost_x_1000": [
            "total_escalated_costs_wo_itcc",
            
            "total_estimated_costs_x_1000_escalated_constant_dollars_od_year",
            "none_4",
            'total_escalated_costs_in_1000s',
            'escalated_cost_in_1000s_note_8',
            'total_estimated_costs_x_1000_escalated_constant_dollars_od_year',
            

        ],
        "estimated_cost_x_1000": [
            "none_3",
            'total_costs_wo_itcc_cab',
       'total_allocated_costs_constant_2020_dollar_in_1000s',
       'total_estimated_costs_x_1000_constant_dollar_2020',
       'total_allocated_costs_constant_2020_dollar_in_1000s',
       'total_estimated_costs_x_1000_constant_dollar_2019',
       'total_estimated_costs_x_1000_constant_dollar_2019',
       'total_estimated_costs_x_1000_constant_dollar_2016',
       'total_estimated_costs_x_1000_constant_dollar_2015',
        
    
       

        ],
        "estimated_time_to_construct": [
            'estimated_time_for_licensing_permitting_construction_months',
            'estimated_time_to_construct_months',
            'estimated_time_to_construct_months2',
            'estimated_time_to_construct_months',
            'estimated_time_to_construct_months',
            'estimated_time_to_construct_months_note_1g',
            'estimated_time_to_construct_months_note_1g',
            'estimated_time_to_construct_months',
            "none_5",
            'estimated_time_to_construct_months',
            'upgrade_duration_months',
            'estimated_time_to_construct_months_note_12', 
             'estimated_time_to_construct_months_note_1g', 
             'estimated_time_to_construct_months_note_1g',
             'estimated_time_to_construct_months_note_345_9_10',
        ],
        "description": ["description"],
        "capacity": [
            "capacity",
            "project size",
            "project mw",
            "mw at poi"
        ],
 
        "max_time_to_construct": [
            'maximum_escalation_duration_months',
            'od_dollar_escalation_duration_months',
            'od_dollar_escalation_duration_months',
            'od_dollar_escalation_duration_months_note1g',
            'od_dollar_escalation_duration_months_note1g',
            "none_6",
            'maximum_escalation_duration_months',
            'od_dollar_escalation_duration_months_note1g', 
            'od_dollar_escalation_duration_months_note1g',
            'potential_duration_months',
            'od_dollar_escalation_duration_months_note_345_9_10',
            
        ]



    }

        # Identify unnamed columns
    unnamed_columns = [col for col in df.columns if pd.isna(col) or col.strip() == "" or col.startswith("Unnamed")]
    if unnamed_columns:
        merge_columns_dict["type_of_upgrade"].extend(unnamed_columns)

    for new_col, old_cols in merge_columns_dict.items():
        existing_cols = [col for col in old_cols if col in df.columns]
        if existing_cols:
            df[new_col] = df[existing_cols].bfill(axis=1).iloc[:, 0]
            cols_to_drop = [col for col in existing_cols if col != new_col]
            if cols_to_drop:
                df.drop(columns=cols_to_drop, inplace=True)

    return df

df = merge_columns(df)
df.drop([ 'none_7',    'unnamed_8',  'interconnection_facilities_costs_x_1000_constant_dollar_2015', 
         'reliability_network_upgrades_costs_x_1000_constant_dollar_2015', 'delivery_network_upgrades_costs_x_1000_constant_dollar_2015', 
         'distribution_upgrades_costs_x_1000_constant_dollar_2015',
         'one_time_costs_b'], axis=1, inplace=True)








 


def reorder_columns(df):
    """
    Reorders the columns of the DataFrame based on the specified order.

    Args:
        df (pd.DataFrame): The DataFrame to reorder.

    Returns:
        pd.DataFrame: The reordered DataFrame.
    """
    desired_order = [
        "q_id",
        "cluster",
        "req_deliverability",
        "latitude",
        "longitude",
        "capacity",
        "point_of_interconnection",
        "type_of_upgrade",
        "upgrade",
        "description",
        "cost_allocation_factor",
        "estimated_cost_x_1000",
        "escalated_cost_x_1000",
 
        "estimated_time_to_construct",
    ]

    # Start with desired columns that exist in the DataFrame
    existing_desired = [col for col in desired_order if col in df.columns]

    # Then add the remaining columns
    remaining = [col for col in df.columns if col not in existing_desired]

    # Combine the two lists
    new_order = existing_desired + remaining

    # Reorder the DataFrame
    df = df[new_order]

    return df        


df= reorder_columns(df)

df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 7/03_raw/ph2_rawdata_cluster7_style_other.csv', index=False)

df = df[~df.apply(lambda row: row.astype(str).isin(["Total Escalated Costs w/o ITCC", "Constant 2016 Dollar in $1000s (Estimate)", "Eastern", "Maximum Cost Responsibility (Network Upgrades)", "Cost Category Note (a - f)"]).any(), axis=1)]
df = df[~df.apply(lambda row: any(str(cell).startswith("Project #:") for cell in row), axis=1)]
df = df[df['type_of_upgrade'].notna() & (df['type_of_upgrade'].astype(str).str.strip() != "")]

 
def process_upgrade_columns(df):
    """
    Given a DataFrame df with a column "type_of_upgrade" that contains both group headers and upgrade data,
    this function:
      1. Inserts a new column "upgrade" as a duplicate of "type_of_upgrade" (placed immediately after it).
      2. Renames rows in "type_of_upgrade" that contain specific phrases as follows:
           - If it contains "Interconnection Facilities", rename to "PTO_IF" (or "PTO_IF Total" if "Total" is present)
           - If it contains "Reliability Network Upgrade", rename to "RNU" (or "RNU Total" if "Total" is present)
           - If it contains "Local Delivery Network Upgrades", rename to "LDNU" (or "LDNU Total" if "Total" is present)
           - If it contains "Area Deliverability Network Upgrades", rename to "ADNU" (or "ADNU Total" if "Total" is present)
           - If it contains "Distribution Upgrades", leave it as is.
      3. Creates a temporary column that only holds the header values (from rows that were detected as header rows) and forward-fills it downward.
         The forward fill stops (i.e. does not fill into a row) if that row’s original "type_of_upgrade" contains any of the "total" indicators.
      4. Replaces "type_of_upgrade" with the forward-filled header values.
      5. Drops the rows that originally were header rows.
      6. This deletes any rows which are either Total or Subtotal or Total cost assigned, the reason is some proejcts have multiple pdfs thus we rather calculate the total in the end.
      
    Returns the updated DataFrame.
    """
    import pandas as pd
    
    # 1. Create a new column "upgrade" immediately after "type_of_upgrade"
    loc = df.columns.get_loc("type_of_upgrade")
    df.insert(loc+1, "upgrade", df["type_of_upgrade"])
    
    # 2. Define a helper to rename header rows.
    def rename_header(val):
        # If the cell contains any of these phrases, rename accordingly.
        # We'll check using the substring test (case-sensitive) per your request.
        
        if "Potential Interconnection Facilities" in val:
            return "Potential PTO_IF" + (" Total" if "Total" in val else "")
        if "Interconnection Facilities" in val:
            return "PTO_IF" + (" Total" if "Total" in val else "")
        elif "Reliability Network Upgrade" in val:
            return "RNU" + (" Total" if "Total" in val else "")
        elif "Local Delivery Network Upgrades" in val:
            return "LDNU" + (" Total" if "Total" in val else "")
        elif "Area Deliverability Network Upgrades" in val:
            return "ADNU" + (" Total" if "Total" in val else "")
        elif "Distribution Upgrades" in val:
            return val  # leave unchanged
        elif "Conditional Assigned Network Upgrades" in val:
            return  ("Total " if "Total" in val else "") + "CANU" 
        elif "Non-Allocated IRNU" in val:
            return  ("Total " if "Total" in val else "") + "Non-Allocated IRNU"
        elif "Non-allocated IRNU" in val:
            return  ("Total " if "Total" in val else "") + "Non-Allocated IRNU"
        
        elif "IRNU-NA" in val:
            return  ("Total " if "Total" in val else "") + "IRNU-NA"
        


        elif "Area Delivery Network Upgrades" in val:
            return "ADNU" + (" Total" if "Total" in val else "")
        else:
            return val
    
    # 3. Identify header rows. We consider a row to be a header row if its "type_of_upgrade" cell 
    # contains any of the target phrases.
    target_phrases = [
        "Interconnection Facilities",
        "Potential Interconnection Facilities" ,
        "Reliability Network Upgrade",
        "Local Delivery Network Upgrades",
        "Area Deliverability Network Upgrades",
        "Distribution Upgrades",
        "Conditional Assigned Network Upgrades",
        "Non-Allocated IRNU",
        "Area Delivery Network Upgrades",
        "Non-allocated IRNU",
        "IRNU-NA",

    ]
    # Create a boolean mask for header rows.
    header_mask = df["type_of_upgrade"].apply(lambda x: any(phrase in x for phrase in target_phrases))
    
    # Apply renaming to the header rows.
    df.loc[header_mask, "type_of_upgrade"] = df.loc[header_mask, "type_of_upgrade"].apply(rename_header)
    
    # 4. Create a temporary column 'header_temp' that holds only the header rows, then forward fill it.
    df["header_temp"] = df["type_of_upgrade"].where(header_mask)
    df["header_temp"] = df["header_temp"].ffill()
    
    # We want to stop the forward fill if we encounter a row that indicates totals.
    # Define a simple function that returns True if a cell contains "Total" or "Subtotal" or "Total cost assigned".
    def is_total_indicator(val):
        return ("Total" in val) or ("Subtotal" in val) or ("Total cost assigned" in val)
    
    # For rows that themselves are total indicators in the "upgrade" column, do not forward-fill (set header_temp to NaN)
    df.loc[df["upgrade"].apply(lambda x: is_total_indicator(x)), "header_temp"] = None
    
    # Now, replace the "type_of_upgrade" column with the forward-filled header
    df["type_of_upgrade"] = df["header_temp"]
    df.drop("header_temp", axis=1, inplace=True)
    
    # 5. Finally, drop the rows that were header rows (i.e. where header_mask is True)
    df = df[~header_mask].reset_index(drop=True)
    
    # Also, drop any rows that have an empty "type_of_upgrade"
    df = df[df["type_of_upgrade"].notna() & (df["type_of_upgrade"].str.strip() != "")]
    
    return df

 

df = process_upgrade_columns(df)


 




 
mappings = {
 "PTO": 'PTO_IF',
 "PNU": "OPNU",
 "PTO's Interconnection Facilities": "PTO_IF",
 "Potential Interconnection Facilities" : "Potential PTO_IF",
 "RNUs, Estimated Costs, and Estimated Time to Construct Summary": "RNU",
 "Non-Allocated IRNU": "RNU",
 "Non-allocated IRNU": "RNU",
    "IRNU-NA": "RNU",
    "Total IRNU-NA": "Total RNU",
 "Total Non-allocated IRNU": "Total RNU",
 "Total Non-Allocated IRNU": "Total RNU",
 }

if 'type_of_upgrade' in df.columns:
  
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: mappings.get(x, x) if isinstance(x, str) else x
    )




df['item'] = df.apply(
    lambda row: 'no' if (
        (pd.notna(row.get('type_of_upgrade')) and 'Total' in str(row['type_of_upgrade'])) or
        (pd.notna(row.get('cost_allocation_factor')) and 'Total' in str(row['cost_allocation_factor']))
    ) else 'yes',
    axis=1
)
   


 
   
 
 


 

 

    
# Step 7: Remove $ signs and convert to numeric
import re

def clean_currency(value):
    """
    Cleans a string by explicitly removing $, *, (Note 2), and similar patterns,
    then converts it to a numeric value.
    """
    if isinstance(value, str):
        # Explicitly remove $, *, and any "(Note ...)"
        value = value.replace('$', '').replace('*', '')
        value = re.sub(r'\(Note \d+\)', '', value)  # Remove patterns like "(Note 2)"
        value = value.replace(',', '').strip()  # Remove commas and extra spaces
    try:
        return pd.to_numeric(value)
    except ValueError:
        return pd.NA  # Return NaN for invalid entries


# Clean the specific columns
for col in ['estimated_cost_x_1000', 'escalated_cost_x_1000']:
    if col in df.columns:
        df[col] = df[col].apply(clean_currency)

# Create Total rows
new_rows = []
columns_to_sum = ['estimated_cost_x_1000', 'escalated_cost_x_1000']
columns_to_populate = ['cluster', 'req_deliverability', 'latitude', 'longitude', 'capacity', 'point_of_interconnection']

for q_id, group in df.groupby('q_id', as_index=False):
    unique_upgrades = group['type_of_upgrade'].dropna().unique()
    for upgrade in unique_upgrades:
        if pd.isna(upgrade):
            continue
        
        rows = group[group['type_of_upgrade'] == upgrade]

                # Check if a Total row already exists for this (q_id, upgrade)
        total_exists = group[
            (group['type_of_upgrade'] == upgrade) & (group['item'] == 'no')
        ].shape[0] > 0
        
        if total_exists:
             
            continue


 
        
        
        if not total_exists:
            # If only one row exists, duplicate it as the total row
            if len(rows) == 1:

                total_row = {col: '' for col in df.columns}  # Initialize all columns as empty strings

                # Populate the necessary fields
                total_row['q_id'] = q_id
                total_row['type_of_upgrade'] = f"Total {upgrade}"
                total_row['item'] = 'no'

                # Populate specified columns from the existing row
                first_row = rows.iloc[0]
                for col in columns_to_populate:
                    if col in df.columns:
                        total_row[col] = first_row[col]

                # Sum the numeric columns (single row, so it remains the same)
                for col in columns_to_sum:
                    if col in rows.columns:
                        total_row[col] = rows[col].sum()
                    else:
                        total_row[col] = 0  # Default to 0 if column is missing

                new_rows.append(total_row)



 
            
            # If multiple rows exist, sum numeric columns and create a total row
            elif len(rows) > 1:
                total_row = {col: '' for col in df.columns}  # Initialize all columns as empty strings

                # Populate the necessary fields
                total_row['q_id'] = q_id
                total_row['type_of_upgrade'] = f"Total {upgrade}"
                total_row['item'] = 'no'

                # Populate the specified columns from the first row in the group
                first_row = rows.iloc[0]
                for col in columns_to_populate:
                    if col in df.columns:
                        total_row[col] = first_row[col]

                # Sum the numeric columns
                for col in columns_to_sum:
                    if col in rows.columns:
                        total_row[col] = rows[col].sum()
                    else:
                        total_row[col] = 0  # Default to 0 if column is missing

                new_rows.append(total_row)
 



      

if new_rows:
    total_rows_df = pd.DataFrame(new_rows)
    for col in df.columns:
        if col not in total_rows_df.columns:
            total_rows_df[col] = None
    total_rows_df = total_rows_df[df.columns]
    df = pd.concat([df, total_rows_df], ignore_index=True)

df.reset_index(drop=True, inplace=True)

# Update 'item' column based on Total in type_of_upgrade or cost_allocation_factor
df['item'] = df.apply(
    lambda row: 'no' if (
        'Total' in str(row.get('type_of_upgrade', '')) or 
        'Total' in str(row.get('cost_allocation_factor', ''))
    ) else 'yes',
    axis=1
)


# Step 7: Move 'item' column next to 'type_of_upgrade'
if 'item' in df.columns and 'type_of_upgrade' in df.columns:
    cols = df.columns.tolist()
    item_index = cols.index('item')
    type_index = cols.index('type_of_upgrade')
    if item_index < type_index:
        cols.insert(type_index + 1, cols.pop(item_index))
    else:
        cols.insert(type_index + 1, cols.pop(item_index))
    df = df[cols]

 



def clean_estimated_time(value):
    if isinstance(value, str):
        value = re.sub(r'(\d+(?:-\w+)*)\s+\w+.*$', r'\1', value, flags=re.IGNORECASE).strip()
    return value

if 'estimated_time_to_construct' in df.columns:
    df['estimated_time_to_construct'] = df['estimated_time_to_construct'].apply(clean_estimated_time)

if 'type_of_upgrade' in df.columns:
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: re.sub(r'\s*\(note \d+\)', '', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x
    )

 
mappings = {
 "PTO": 'PTO_IF',
 "PNU": "OPNU",
 "PTO's Interconnection Facilities": "PTO_IF",
 "RNUs, Estimated Costs, and Estimated Time to Construct Summary": "RNU",
'Total PTO_IF': 'PTO_IF',
'PTO_IF Total': 'PTO_IF',
 'Total RNU': 'RNU',
 'RNU Total': 'RNU',
 'LDNU Total': 'LDNU',
 'Total LDNU': 'LDNU',
 'Total OPNU' : 'OPNU',
 'Total CANU': 'CANU',
 'Total LOPNU': 'LOPNU',
 'Total ADNU': 'ADNU',
 "Potential Interconnection Facilities" : "Potential PTO_IF",
 "Total Potential Interconnection Facilities" : "Potential PTO_IF",
    'Total Potential PTO_IF': 'Potential PTO_IF',
 'Total Distribution Upgrades': 'Distribution Upgrades',
 'Distribution Upgrades Total': 'Distribution Upgrades',
 "Non-allocated IRNU": "RNU",
    "IRNU-NA": "RNU",
    "Total IRNU-NA": "Total RNU",
 "Total Non-allocated IRNU": "Total RNU",
}

if 'type_of_upgrade' in df.columns:
  
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: mappings.get(x, x) if isinstance(x, str) else x
    )



 

if 'type_of_upgrade' in df.columns:
    previous_type_of_upgrade = None
    for i in range(len(df)):
        if df.at[i, 'type_of_upgrade'] == 'Total':
            if previous_type_of_upgrade is not None:
                df.at[i, 'type_of_upgrade'] = previous_type_of_upgrade
        else:
            previous_type_of_upgrade = df.at[i, 'type_of_upgrade']

numeric_columns = [
    
    'estimated_cost_x_1000',
    'estimated_time_to_construct',
     
    'adnu_cost_rate_x_1000',
    'escalated_cost_x_1000',
     
]
non_numeric_columns = ['type_of_upgrade', 'upgrade', 'description']

for col in non_numeric_columns:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: 'None' if pd.isna(x) else x)

for col in numeric_columns:
    if col in df.columns:
        df[col] = df[col].replace('-', pd.NA)
        df[col] = df[col].fillna(0)

if 'original_order' in df.columns and 'q_id' in df.columns:
    df['original_order'] = df.index
    df = df.sort_values(by=['q_id', 'original_order'], ascending=[True, True])
    df = df.drop(columns=['original_order'])


def reorder_columns(df):
    """
    Reorders the columns of the DataFrame based on the specified order.

    Args:
        df (pd.DataFrame): The DataFrame to reorder.

    Returns:
        pd.DataFrame: The reordered DataFrame.
    """
    desired_order = [
        "q_id",
        "cluster",
        "req_deliverability",
        "latitude",
        "longitude",
        "capacity",
        "point_of_interconnection",
        "type_of_upgrade",
        "upgrade",
        "description",
        "cost_allocation_factor",
        "estimated_cost_x_1000",
        "escalated_cost_x_1000",
        "total_estimated_cost_x_1000",
        "total_estimated_cost_x_1000_escalated",
        "estimated_time_to_construct",
    ]

    # Start with desired columns that exist in the DataFrame
    existing_desired = [col for col in desired_order if col in df.columns]

    # Then add the remaining columns
    remaining = [col for col in df.columns if col not in existing_desired]

    # Combine the two lists
    new_order = existing_desired + remaining

    # Reorder the DataFrame
    df = df[new_order]

    return df        


df= reorder_columns(df)


# Save itemized and totals separately
if 'item' in df.columns:
    itemized_df = df[df['item'] == 'yes']
 
     
    itemized_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 7/02_intermediate/costs_phase_2_cluster_7_style_others_itemized.csv', index=False)

    totals_columns = ['upgrade', 'description', 'cost_allocation_factor', 'estimated_time_to_construct']
    existing_totals_columns = [col for col in totals_columns if col in df.columns]
    totals_df = df[df['item'] == 'no'].drop(columns=existing_totals_columns, errors='ignore')
    # Define the cost columns.
    cost_cols = ['estimated_cost_x_1000', 'escalated_cost_x_1000']

    # Build an aggregation dictionary:
    # For columns not in grouping or cost_cols, we assume they are identical and take the first value.
    agg_dict = {col: 'first' for col in totals_df.columns 
                if col not in ['q_id', 'type_of_upgrade'] + cost_cols}
    # NEW: only add those cost_cols that are present
    agg_dict.update({
        col: 'sum' 
        for col in cost_cols 
        if col in totals_df.columns
    })

    # (Optional) Diagnostic print to confirm keys line up:
    print("Will aggregate these columns:", agg_dict)


    

    # Group by both q_id and type_of_upgrade using the aggregation dictionary.
    totals_df = totals_df.groupby(['q_id', 'type_of_upgrade'], as_index=False).agg(agg_dict)
    totals_df = reorder_columns(totals_df)
    totals_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 7/02_intermediate/costs_phase_2_cluster_7_style_others_total.csv', index=False)

print(f"Itemized rows saved to 'costs_phase_2_cluster_14_itemized.csv'.")
print(f"Filtered Total rows saved to 'costs_phase_2_cluster_14_total.csv'.")



if 'type_of_upgrade' in df.columns:
    print(df['type_of_upgrade'].unique())

if 'q_id' in df.columns:
    print(df['q_id'].unique())

if 'cluster' in df.columns:
    print(df['cluster'].unique())



Will aggregate these columns: {'cluster': 'first', 'req_deliverability': 'first', 'latitude': 'first', 'longitude': 'first', 'capacity': 'first', 'point_of_interconnection': 'first', 'item': 'first', 'max_time_to_construct': 'first', 'estimated_cost_x_1000': 'sum', 'escalated_cost_x_1000': 'sum'}
Itemized rows saved to 'costs_phase_2_cluster_14_itemized.csv'.
Filtered Total rows saved to 'costs_phase_2_cluster_14_total.csv'.
['RNU' 'PTO_IF' 'LDNU' 'Distribution Upgrades']
[1064 1065 1066 1069 1070 1071 1074 1076]
[7]


/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_67294/161967618.py:229: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[new_col] = df[existing_cols].bfill(axis=1).iloc[:, 0]
/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_67294/161967618.py:229: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[new_col] = df[existing_cols].bfill(axis=1).iloc[:, 0]
/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_67294/161967618.py:229: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change i

## Addendums - NO ADDENUMS in this case.

# Column Names

In [ ]:
import pandas as pd
import re
import unicodedata

# Load the CSV file
df = pd.read_csv("/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 7/03_raw/ph2_rawdata_cluster7_style_others_addendums.csv", dtype={'estimated_time_to_construct': str})

df['q_id'] = df['q_id'].astype('Int64')
df['cluster'] = df['cluster'].astype('Int64')


def clean_column_headers(headers):
    """Cleans column headers by normalizing and removing unwanted characters."""
    cleaned_headers = []  # Initialize an empty list to hold the cleaned header names.
    for header in headers:  # Iterate over each header in the input.
        if header is None:
            header = ""  # If the header is None, set it to an empty string.
        elif isinstance(header, str):  # Otherwise, if the header is a string:
            #header = header.lower()  # Convert the header to lowercase.
            header = re.sub(r'\s+', ' ', header)  # Replace one or more whitespace characters with a single space.
            #header = re.sub(r'\(.*?\)', '', header)  # Remove any text within parentheses (non-greedy).
            header = re.sub(r'[^a-zA-Z0-9\s()/+=_]', '', header)  # Remove any character that is not a letter, number, or whitespace.
            header = header.strip()  # Remove any leading or trailing whitespace.
        cleaned_headers.append(header)  # Append the cleaned header to the list.
    return cleaned_headers  # Return the list of cleaned headers.



df.columns = clean_column_headers(df.columns)

def convert_to_snake_case(column_name):
    column_name = column_name.strip().lower()
    column_name = re.sub(r'[\s\-]+', '_', column_name)
    column_name = re.sub(r'[^\w]', '', column_name)
    return column_name

def clean_string_cell(value):
    if isinstance(value, str):
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
        value = value.replace('\n', ' ').strip()
    return value

df = df.map(clean_string_cell)
df.columns = [convert_to_snake_case(col) for col in df.columns]
print("After cleaning:", df.columns.tolist())

# It

# Merge- only update

In [10]:
df = pd.read_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 7/02_intermediate/costs_phase_2_cluster_7_style_others_itemized.csv')
df2 = pd.read_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 7/02_intermediate/costs_phase_2_cluster_7_style_others_total.csv')

df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 7/01_clean/costs_phase_2_cluster_7_style_others_itemized_updated.csv', index=False)
df2.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 7/01_clean/costs_phase_2_cluster_7_style_others_total_updated.csv', index=False)